pip install numpy  
pip install pandas  
pip install openpyxl  
pip install tqdm
pip install ffmpeg  
pip install pydub  
conda install -c conda-forge librosa  
pip install -U praat-parselmouth
pip install lime
pip install  
python3 -m spacy download fr_dep_news_trf  
pip install intel-openmp  
pip install spacy-transformers

Please restart the kernel after installing everything

In [1]:
import pandas as pd
import pydub
from tqdm import tqdm
import os
from utils import get_end_from_start, get_start_end_from_file
from data import read_interview

pydub.AudioSegment.converter = r"C:/Users/Kinza/anaconda3/envs/pie/Library/bin/ffmpeg.exe" #CHANGE THIS

In [2]:
def load_interviews(video_folder,startend_file) :
    '''Loads the audios from each interview question.
    Arguments:
        video_folder : str. The name of the folder containing mp4 videos.
        startend_file : str. The name of the file containing the video informations. 
            Must contain columns 'mail' and 'time'.
    Returns a list of Interview objects
    '''
    filenames = tqdm(os.listdir(video_folder))
    df_startend = get_start_end_from_file(startend_file)
    audios = list(map(lambda f : read_interview(video_folder,df_startend,f), filenames))
    return [item for sublist in audios for item in sublist]

In [3]:
video_folder = 'videos/'
df_name = 'notes_entretiens_all.xlsx'

interviews = load_interviews(video_folder,df_name)

  0%|          | 0/25 [00:00<?, ?it/s]

alexis.przybylak@student.isae-supaero.fr.mp4


C:\Users\Kinza\anaconda3\envs\pie\lib\site-packages\librosa\core\convert.py:1350: RuntimeWarning: divide by zero encountered in log10
  + 2 * np.log10(f_sq)


 Bonjour dernière année d'école d'ingénieur en gros c'est une école spécialisée dans le monde est le deuxième année de basket et une prépa ne sais pas à Paris je suis la sœur de l'univers et le domaine opération de paiement donc donc avec des cours et que ramadan est général l'environnement rappel à propos encore les télécommunications et le traitement d'image et pour le super héros je ne se réaliser de stage sur la propulsion électrique et le deuxième à l'érable à Toulouse sur le champ magnétique du soleil et en ce qui concerne le parcours extrascolaire en 1re année 2e année l'euro est également dans un groupe guitariste dans un groupe de musique toujours en sport


C:\Users\Kinza\anaconda3\envs\pie\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


...

basile.rochut@student.isae-supaero.fr.mp4
 Bonjour je m'appelle Basile refus je suis étudiant en dernière année de cycle ingénieur Hallyday super-héros à Toulouse
 je pense que dans ces valeurs celle qui me correspond le plus serein parents et finalement je suis rendu compte lorsque je devais malade j'ai 7 équipes que plus important c'était que tout le monde arrive à se sentir à l'aise dans les tâches qui lui était confiée et que bah tout seul on peut pas tout faire mais ça c'était peut-être un défaut que j'avais auparavant et je me suis rendu compte de l'importance du travail d'équipe et je m'en rends compte également rendu compte également grâce à un message sur mon stage de césure j'ai travaillé sur le Vendée Globe j'étais avec mon tuteur et un autre employé nous travaillons sur le développement de l'automatisation et il va tout seul et il va tout seul impossible réaliser la tâche c'est le travail d'équipe une émulation qui était très importante
 par une fois où j'ai dû prendre une 

 12%|█▏        | 3/25 [07:40<50:38, 138.10s/it]  

boudabousarah@gmail.com.mp4
 en relations internationales


 12%|█▏        | 3/25 [07:42<56:33, 154.23s/it]


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import FrenchStemmer
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words

class FrenchStemTokenizer(object):

    def __init__(self, remove_non_words=False):
        self.st = FrenchStemmer()
        self.stopwords = set(stopwords.words('french'))
        self.words = set(words.words())
        self.remove_non_words = remove_non_words

    def __call__(self, doc):
        # tokenize words and punctuation
        word_list = wordpunct_tokenize(doc)
        # remove stopwords
        word_list = [word for word in word_list if word not in self.stopwords]
        # remove non words
        if (self.remove_non_words):
            word_list = [word for word in word_list if word in self.words]
        # remove 1-character words
        word_list = [word for word in word_list if len(word) > 1]
        # remove non alpha
        word_list = [word for word in word_list if word.isalpha()]
        return [self.st.stem(t) for t in word_list]

_speech = "Bonjour je m'appelle Basile refus je suis étudiant en dernière année de cycle ingénieur Hallyday super-héros à Toulouse je pense que dans ces valeurs celle qui me correspond le plus serein parents et finalement je suis rendu compte lorsque je devais malade j'ai 7 équipes que plus important c'était que tout le monde arrive à se sentir à l'aise dans les tâches qui lui était confiée et que bah tout seul on peut pas tout faire mais ça c'était peut-être un défaut que j'avais auparavant et je me suis rendu compte de l'importance du travail d'équipe et je m'en rends compte également rendu compte également grâce à un message sur mon stage de césure j'ai travaillé sur le Vendée Globe j'étais avec mon tuteur et un autre employé nous travaillons sur le développement de l'automatisation et il va tout seul et il va tout seul impossible réaliser la tâche c'est le travail d'équipe une émulation qui était très importante par une fois où j'ai dû prendre une cette que j'ai en tête ce serait départ et en leur donnant toutes les données récupérer la vie je me suis rendu compte que les erreurs qu'on trouve dans mon fils et développement non pas demandé des résultats de la qui me faisait une erreur dans leur pays et finalement c'est plutôt bien déroulé plus que je les ai aidés à résoudre ce problème là et ça a été notifié par l'entreprise à mon tuteur qui était très ravi que"

countvect = CountVectorizer()
text_fts = countvect.fit_transform([_speech])

count_array = text_fts.toarray()

_words = list(countvect.get_feature_names_out())
word_count = count_array.sum()

countvect = CountVectorizer(tokenizer=FrenchStemTokenizer(remove_non_words=False))
text_fts = countvect.fit_transform([_speech])

_vec = list(countvect.get_feature_names_out())

_vec

True


['aid',
 'ais',
 'anné',
 'appel',
 'arriv',
 'auparav',
 'automatis',
 'autr',
 'bah',
 'basil',
 'bien',
 'bonjour',
 'cel',
 'cet',
 'compt',
 'confi',
 'correspond',
 'cycl',
 'césur',
 'demand',
 'derni',
 'dev',
 'don',
 'défaut',
 'départ',
 'déroul',
 'développ',
 'dû',
 'emploi',
 'entrepris',
 'erreur',
 'fair',
 'fais',
 'fil',
 'final',
 'fois',
 'glob',
 'grâc',
 'hallyday',
 'héros',
 'import',
 'impossibl',
 'ingénieur',
 'lorsqu',
 'là',
 'malad',
 'messag',
 'mond',
 'non',
 'notifi',
 'où',
 'parent',
 'pay',
 'pens',
 'peut',
 'plus',
 'plutôt',
 'prendr',
 'problem',
 'rav',
 'refus',
 'rend',
 'rendu',
 'réalis',
 'récuper',
 'résoudr',
 'résultat',
 'sent',
 'serein',
 'seul',
 'stag',
 'sup',
 'toulous',
 'tout',
 'travail',
 'travaill',
 'travaillon',
 'tres',
 'trouv',
 'tuteur',
 'tâch',
 'têt',
 'va',
 'valeur',
 'vend',
 'vi',
 'ça',
 'égal',
 'émul',
 'équip',
 'étudi',
 'être']